In [1]:
from sklearn.metrics import roc_auc_score
import pandas as pd
import random
from tqdm import tqdm
import os
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
target_case3 = pd.read_csv('/content/drive/MyDrive/Хакатон Альфа-банка. Данные для организаторов/targets/targets_case3.csv')

In [ ]:
path = '/content/drive/MyDrive/DOS-тестирование платформы/2. Стресс-тест №2/Кейс 3. Предсказания'
files = os.listdir(path)

In [ ]:
def metric_calculation(submission, target=target_case3):
  submission = submission[['id', 'target']]
  target = target[['id', 'target']]

  result = submission.merge(target, left_on=['id'], right_on=['id'])

  prediction = result.iloc[:, 1]
  y_true = result.iloc[:, 2]

  mtrc = max(roc_auc_score(y_true=y_true, y_score=prediction), 1-roc_auc_score(y_true=y_true, y_score=prediction))

  return mtrc

In [ ]:
# Создадим множество для записи метрик
metrics = set()
# Создадим список для записи файлов, с дублирующимися метриками
bad_files = set()

In [ ]:
for file in files:
  data = pd.read_csv(path+f'/{file}')
  metric = metric_calculation(submission=data)
  if metric not in metrics:
    metrics.add(metric)
  else:
    bad_files.add(file)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for bad_file in bad_files:
  data = pd.read_csv(path+f'/{bad_file}')

  for i in tqdm(range(data.shape[0])):
    data['target'].iloc[i] = data['target'].iloc[i] + random.random()

  data.to_csv('/content/drive/MyDrive/DOS-тестирование платформы/2. Стресс-тест №2/Обновленные файлы 3'+f'/{bad_file}', index=False)

100%|██████████| 171919/171919 [01:26<00:00, 1999.05it/s]


In [ ]:
predictions = '/content/drive/MyDrive/DOS-тестирование платформы/2. Стресс-тест №2/Обновленные файлы 3'
target_case3 = pd.read_csv('/content/drive/MyDrive/Хакатон Альфа-банка. Данные для организаторов/targets/targets_case3.csv')
pred_files = os.listdir(predictions)

In [ ]:
mtrc = []

for file in pred_files:
  data = pd.read_csv(predictions+f'/{file}')
  metric = metric_calculation(submission=data)
  mtrc.append(metric)

print('Число собранных метрик -', len(mtrc))
print('Чисто уникальных предсказаний -', len(set(mtrc)))

Число собранных метрик - 87
Чисто уникальных предсказаний - 87


In [ ]:
print(min(mtrc))